# Hopf Bifurcation Study

In [ ]:
from tqdm.notebook import tqdm
from os.path import isfile, splitext

import h5py
import numpy as np
import jax
from jax import numpy as jnp
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import dolfin as dfn

from femvf import load, statefile as sf, meshutils
from femvf.models.transient import solid as smd, fluid as fmd
from femvf.signals.solid import make_sig_glottal_width_sharp
from blocktensor import h5utils as bh5utils
from vfsig import modal
from vfsig.misc import resample_over_uniform_period

import libhopf
import main_hopf
import libsignal
from lib_main_transient import case_config
import h5utils
from postprocutils import postprocess_case_to_signal

In [ ]:
## Global vars
ZETA = 1e-4
R_SEP = 1.0
Y_GAP = 1e-2
# PSUBS = np.concatenate([np.arange(200, 300, 10), np.arange(300, 1000, 100)])*10
PSUBS = np.arange(550, 650, 10) * 10 
ECOV = 5e3 * 10
# EBODY = 15e3 * 10
EBODY = 5e3 * 10

# Load the transient model
INIT_STATE_TYPE = 'static'

OUT_DIR = f'out/zeta{ZETA:.2e}_rsep{R_SEP:.1f}_ygap{Y_GAP:.2e}_init{INIT_STATE_TYPE}_fixed_rsep'
FluidType = fmd.BernoulliMinimumSeparation
# OUT_DIR = f'out/zeta{ZETA:.2e}_rsep{R_SEP:.1f}_ygap{Y_GAP:.2e}_init{INIT_STATE_TYPE}_variable_rsep'
# FluidType = fmd.Bernoulli

MESH_NAME = 'BC-dcov5.00e-02-cl1.00'
mesh_path = f'mesh/{MESH_NAME}.xml'
model_tran = load.load_transient_fsi_model(
    mesh_path, None, 
    SolidType=smd.KelvinVoigt, 
    FluidType=FluidType, 
    coupling='explicit'
)

# load the Hopf models
res, dres = main_hopf.setup_models(mesh_path)
_region_to_dofs = meshutils.process_celllabel_to_dofs_from_forms(
    res.solid.forms, res.solid.forms['fspace.scalar'])
_props = main_hopf.set_props(res.props.copy(), _region_to_dofs, res)
res.set_props(_props)
dres.set_props(_props)

res_hopf = libhopf.HopfModel(res, dres)

## Post-process nonlinear oscillations (transient simulations)

In [ ]:
## Initialize functions to extract glottal width from saved simulations

# Number of points per period to use in processing
N_PER_PERIOD = 100
proc_gw_tran = make_sig_glottal_width_sharp(model_tran)
proc_gw_hopf = libsignal.make_glottal_width(res_hopf, num_points=N_PER_PERIOD)
def time(f):
    return np.array(f.get_times())

In [ ]:
## Post processing transient simulations
postprocess_fname = f'{OUT_DIR}/data.h5'
signal_to_proc = {
    'glottal_width': proc_gw_tran,
    'time': time}
files = [f'{OUT_DIR}/{case_config(MESH_NAME, psub, ECOV, EBODY)}.h5' for psub in PSUBS]

# `SIGNALS` is a dictionary mapping each `case` and signal name to a time signal
SIGNALS = postprocess_case_to_signal(postprocess_fname, files, model_tran, signal_to_proc)

# with h5py.File(postprocess_fname, mode='r') as f:
#     SIGNALS = h5utils.h5_to_dict(f, {})

In [ ]:
## Resample the transient simulations so that they a fixed number of points per period
cases = list(set([key.split('/')[0] for key in SIGNALS.keys()]))

# NOFFSET controls the portion of the signal to treat as 'steady state'
# -8124, sets the final 8124 samples to be considered a steady state portion
# This portion of the signal is resampled on a periodic basis
NOFFSET = -2**10
NPERIOD = 5


MEASURES = {}
for case in cases:
    # F0 and period are in units of samples
    fund_freq, _, df = modal.estimate_fundamental_mode(
        SIGNALS[f'{case}/glottal_width'][NOFFSET:]
    )
    MEASURES[f'{case}/fund_freq'] = fund_freq
    MEASURES[f'{case}/dfreq'] = df
    
    MEASURES[f'{case}/period'] = fund_freq**-1

RESIGNALS = {}
for case in cases:
    xp = np.arange(SIGNALS[f'{case}/glottal_width'][NOFFSET:].size)
    fp = SIGNALS[f'{case}/glottal_width'][NOFFSET:]
    
    # `num_period` is the number of periods/cycles in the sequence
    # This should always be an integer due to how FFT computes the
    # signal frequencies
    samples_per_period = MEASURES[f'{case}/period']
    num_period = int(round(fp.size/samples_per_period))
    
    x = np.linspace(0, num_period, int(num_period)*N_PER_PERIOD + 1) * samples_per_period
    
    RESIGNALS[f'{case}/glottal_width'] = np.interp(x, xp, fp)

resignals_update = {
    f'{case}/time': 
    np.arange(RESIGNALS[f'{case}/glottal_width'].size)/MEASURES[f'{case}/period']
    for case in cases}

RESIGNALS.update(resignals_update)

In [ ]:
## Post processing Hopf simulation (load the Hopf bifurcation state)
hopf_sim_fpath = f"out/hopf_state.h5"

with h5py.File(hopf_sim_fpath, mode='r') as f:
    hopf_state = bh5utils.read_block_vector_from_group(f)

# Decompose the Hopf state vector to common components
xfp_hopf = hopf_state[:4]
mode_hopf_real, mode_hopf_imag = hopf_state[4:8], hopf_state[8:12]
# mode_hopf = mode_hopf_real + 1j*mode_hopf_imag
omega_hopf = hopf_state['omega'][0]
psub_hopf = hopf_state['psub'][0]

unit_mode_real, unit_mode_imag = libhopf.normalize_eigenvector_amplitude(
    mode_hopf_real, mode_hopf_imag)

## Plots

### Nonlinear oscillations

In [ ]:
## Plot transient simulations
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

for psub in PSUBS:
    case = case_config(MESH_NAME, psub, ECOV, EBODY)
    ret = RESIGNALS[f'{case}/time']
    regw = RESIGNALS[f'{case}/glottal_width']
    
    t = SIGNALS[f'{case}/time']
    dt = t[1] - t[0]
    freq = MEASURES[f'{case}/fund_freq'] / dt
    dfreq = MEASURES[f'{case}/dfreq'] / dt
    
    # t = SIGNALS[f'{case}/time']
    # gw = SIGNALS[f'{case}/glottal_width']
    
    ax.plot(ret, regw, label=f"$p_{{sub}}$: {psub/10:.1f} Pa $F_0$: {freq:.1f} +- {dfreq:.1f} Hz", lw=1.0)
    
# ax.set_xlim(60, 70)
# ax.set_xlim(0, 2)

ax.set_xlabel("Time [period]")
ax.set_ylabel("Glottal width [cm]")
ax.legend(loc='lower left', bbox_to_anchor=(0, 1))

fig.tight_layout()
fig.savefig(f'fig/transient_glottal_width.png')

### Small amplitude oscillations

In [ ]:
## Plot Hopf simulations
fig, ax = plt.subplots(1, 1)

for ampl in np.linspace(0, 20000.0, 5):
    gw = proc_gw_hopf(
        hopf_state.to_ndarray(),
        np.array([ampl, 0.0]))
    ax.plot(gw, label=f"Amplitude {ampl:.2e}")
    
ax.set_xlabel(f"Time [period]")
ax.set_ylabel("Glottal width [cm]")
ax.legend()

fig.tight_layout()
fig.savefig("fig/onset_gw_vs_amplitude.png", dpi=250)

In [ ]:
## Plot the onset mode shape
import dolfin as dfn
XREF = res.solid.XREF.vector()

NPHASE = 5
phases = 2*np.pi*np.linspace(0, 1, NPHASE+1)[:-1]

VERT_TO_VECDOF = dfn.vertex_to_dof_map(res.solid.forms['fspace.vector'])
CELLS = res.solid.forms['mesh.mesh'].cells()

fig, axs = plt.subplots(1, NPHASE, sharey=True, sharex=True, figsize=(10, 5))
for ax, phase in zip(axs, phases):
    ampl = 5e4*np.exp(1j*phase)
    mode_hopf_dof_order = (
        XREF[:] + hopf_state['u'] 
        + np.real(
            ampl * (
                hopf_state['u_mode_real'] 
                + 1j*hopf_state['u_mode_imag']
            )
        )
    )
    
    mode_hopf_vert_order = mode_hopf_dof_order[VERT_TO_VECDOF].reshape(-1, 2)
    # print(mode_hopf_vert_order.shape)
    ax.triplot(*mode_hopf_vert_order.T, triangles=CELLS)
    ax.set_title(f"${phase/np.pi:.2f}\pi$ rad")
    
for ax in axs:
    ax.set_aspect('equal')
    ax.set_xlabel("x [cm]")
axs[0].set_ylabel("y [cm]")
    
fig.tight_layout()
fig.savefig(f'fig/hopf_mode_shape.png', dpi=200)

### Compare nonlinear and small-amplitude oscillations

In [ ]:
## Plot Transient vs Hopf simulations
SCALE = np.array([1e4, 1])

def sqr_err(sc_ampl_phase, gw_ref):
    """
    Return the squared error between small amplitude and reference glottal widths
    """
    ampl_phase = sc_ampl_phase*SCALE
    gw_hopf = proc_gw_hopf(
        hopf_state.to_ndarray(), 
        ampl_phase)
    
    # Weight the reference glottal width measurements
    # linearly with glottal width; larger glottal widths
    # get higher weights
    weight = gw_ref/jnp.abs(gw_ref.max()) 
    
    err = weight*(gw_hopf-gw_ref)**2
    return jnp.sum(err)

from scipy import optimize

fig, ax = plt.subplots(1, 1)

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
psubs = np.array([600, 700, 800, 900])*10
psubs = PSUBS
# psubs = PSUBS[0::2]

cases = [case_config(MESH_NAME, psub, ECOV, EBODY) for psub in psubs]
for ii, (psub, color, case) in enumerate(zip(psubs, colors, cases)):
    # For each case, load the transient simulation glottal width 
    t = RESIGNALS[f'{case}/time'][-N_PER_PERIOD:]
    gw_tran = RESIGNALS[f'{case}/glottal_width'][-N_PER_PERIOD:]
    
    # Calculate an optimal [phase, amplitude] for the onset glottal width
    # to minimize the difference from the onset glottal width
    ampl_phase0 = np.array([5e3, 0.0])
    opt_res = optimize.minimize(sqr_err, ampl_phase0/SCALE, args=(gw_tran,), jac=jax.grad(sqr_err, 0), method='BFGS')
    ampl_phase = opt_res['x'] * SCALE
    # print(opt_res)
    
    # Compute the onset glottal width with the optimal phase and amplitude
    gw_hopf0 = proc_gw_hopf(
        hopf_state.to_ndarray(),
        ampl_phase0)
    
    gw_hopf = proc_gw_hopf(
        hopf_state.to_ndarray(),
        ampl_phase)
    
    ax.plot(t, gw_tran, color=color, ls='-')
    ax.plot(t, gw_hopf, color=color, ls='-.')
    ax.plot(t, gw_hopf0, color=color, ls=':')
    
ax.set_xlabel("t/T")
ax.set_ylabel("Glottal width [cm]")

from matplotlib.lines import Line2D

f0s = []
for case in cases:
    _t = SIGNALS[f'{case}/time']
    dt = _t[1]-_t[0]
    f0s.append((MEASURES[f'{case}/period'] * dt)**-1)

# Make a legend describing the relationship 
# ls -> model type
# psub -> color
_model_type_ls = ['-', '-.']
_model_type_lines = [Line2D([0], [0], color='k', ls=ls) for ls in _model_type_ls] 
_model_type_labels = ["Transient", f"Onset {omega_hopf/2/np.pi:.1f} Hz {psub_hopf/10:.1f} Pa"]

_psub_colors = colors[:len(psubs)]
_psub_lines = [Line2D([0], [0], color=color) for color in _psub_colors]
_psub_labels = [f"{psub/10:.0f} Pa, {f0:.1f} Hz" for psub, f0 in zip(psubs, f0s)]
ax.legend(
    _model_type_lines+_psub_lines,
    _model_type_labels+_psub_labels,
    loc=(0, 1)
)

fig.savefig('fig/transient_vs_onset_gw.png', dpi=200)

### Optimization results

In [ ]:
# Load mesh and DOF map information for plotting data on the mesh
TRI = res.solid.forms['mesh.mesh'].coordinates()
X, Y = TRI[:, 0], TRI[:, 1]
CELLS = res.solid.forms['mesh.mesh'].cells()

# These indices select DOFs from a scalar function in vertex-order
IDX_VERT = dfn.vertex_to_dof_map(res.solid.forms['fspace.scalar'])

In [ ]:
# Load optimization history
with h5py.File('out/opt_hist.h5', mode='r') as f:
    objs = f['objective'][:]
    grads = [bh5utils.read_block_vector_from_group(f['grad'], nn) for nn in range(len(objs))]
    params = [bh5utils.read_block_vector_from_group(f['parameters'], nn) for nn in range(len(objs))]
    
its = np.arange(len(objs))

#### Plot a summary of optimization results

In [ ]:
# Indicate a specific iteration to plot parameters for
N = -4
grad = grads[N]
param = params[N]

In [ ]:
print(param.bshape)
print(res.props.bshape)

In [ ]:
# First you have to compute the spectrum; if you save more data in the optimization history file
# you might be able to avoid some of these computations

# Set the dynamical system residual properties + subglottal pressure
res.set_props(param[:-2])
res.control['psub'].set(objs[N])
res.set_control(res.control)

x_fp0 = res.state.copy()
x_fp0.set(0.0)
x_fp, info = libhopf.solve_fixed_point(res, x_fp0)
print(info)
omegas, *_eigvecs = libhopf.solve_linear_stability(res, x_fp)

print(omegas)

In [ ]:
objs_pa = objs/10

fig = plt.figure(figsize=(10, 6))
gs = gridspec.GridSpec(3, 2, figure=fig)

# Plot the objective function history
ax_obj = fig.add_subplot(gs[0, :])
ax = ax_obj

ax.plot(objs_pa)
ax.axvline([its[N]], color='k', ls='-.', alpha=0.5)

ax.set_xlabel("Iteration")
ax.set_ylabel("Onset pressure [Pa]")
ax.set_xlim(its[[0, -1]] + np.array([-1, 1]))
ax.set_ylim([0.8*objs_pa.min(), 1.2*objs_pa[0]])

# Plot the current parameter guess
ax_mod = fig.add_subplot(gs[1, 0])
ax = ax_mod

mappable = ax.tripcolor(X, Y, CELLS, param['emod'][IDX_VERT]/10/1e3)
ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")

cbar = fig.colorbar(mappable, ax=ax)
cbar.ax.set_ylabel("$E$ [kPa]")
ax.set_aspect(1)

# Plot the current gradient
ax_grad = fig.add_subplot(gs[1, 1])
ax = ax_grad

mappable = ax.tripcolor(X, Y, CELLS, grad['emod'][IDX_VERT]/10/1e3)
ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")

cbar = fig.colorbar(mappable, ax=ax)
cbar.ax.set_ylabel("$dE$ [Pa/kPa]")
ax.set_aspect(1)

# Plot the spectrum
ax_spec = fig.add_subplot(gs[2, 0])
ax = ax_spec
ax.scatter(omegas.real, omegas.imag)
ax.axhline(0, color='k')
ax.axvline(0, color='k')

ax.set_xlabel("Real")
ax.set_ylabel("Imag")
ax.set_xlim(-100, 50)

fig.tight_layout()